# 20 - Race Condition

https://adventofcode.com/2024/day/20


In [1]:
// Imports

import colors from "../../utils/colors.ts"
import objects from "../../utils/objects.ts"
import strings from "../../utils/strings.ts"
import numbers from "../../utils/numbers.ts"
import plots from "../../utils/plots.ts";
import images from "../../utils/images.ts";
import arrays from "../../utils/arrays.ts";

In [2]:
// Read Input

const file = await Deno.readTextFile("input-base.txt");
const area = file.split("\n").map((row) => row.split(""));

arrays.print2D(area)

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############


In [3]:
// Prepare Data

const findIndex = (target: string, area: string[][]) => {
    for (let i = 0; i < area.length; i++) {
        for (let j = 0; j < area[i].length; j++) {
            if (area[i][j] === target) return [i, j];
        }
    }
}

const visitedKey = (i: number, j: number) => `${i}-${j}`;
const isWall = el => el === "#";
const directions = [[0, 1], [1, 0], [0, -1], [-1, 0]];


[findIndex("S", area), findIndex("E", area)]

[ [ 3, 1 ], [ 7, 5 ] ]

In [4]:
// Part 1 - How many cheats would save you at least x picoseconds?

const findFastestPath = (area: string[][], maxIterations = 1000000) => {
    const startPosition = findIndex("S", area);
    const endPosition = findIndex("E", area);

    const stack = [{ position: startPosition, path: new Set<string>() }];

    const visited = {}

    while (stack.length > 0) {
        const { position, path } = stack.shift();
        // console.log(stack);
        const [i, j] = position;

        if (i < 0 || i >= area.length || j < 0 || j >= area[0].length) {
            continue;
        }

        if (maxIterations-- < 0) {
            console.log('Max Iterations Reached');
            break; // Just to avoid accidental infinite/long running loop
        }

        const currentPathKey = visitedKey(i, j);
        if (visited[currentPathKey]) continue;
        visited[currentPathKey] = true;

        path.add(currentPathKey);
        if (i === endPosition[0] && j === endPosition[1]) {
            return path;
        }

        directions.forEach((d) => {
            const next = area[i + d[0]] ? area[i + d[0]][j + d[1]] : null;
            if (next && !isWall(next)) {
                stack.push({ position: [i + d[0], j + d[1]], path: new Set([...path]) });
            }
        });
    }
    return null;
}

const validCheat = (a: number[], b: number[], area: string[][]) => {
    const distance = Math.abs(a[0] - b[0]) + Math.abs(a[1] - b[1]);
    if (distance !== 2) return false;
    if (a[0] === b[0]) {
        const mid = area[a[0]][(a[1] + b[1]) / 2];
        return isWall(mid);
    }
    if (a[1] === b[1]) {
        const mid = area[(a[0] + b[0]) / 2][a[1]];
        return isWall(mid);
    }
    return false;
}

const findCheats = (area: string[][], path: Set<string>) => {
    const cheats = [];

    const pathArray = [...path].map((p) => p.split("-").map(Number));
    for (let i = 0; i < pathArray.length - 1; i++) {
        for (let j = i + 1; j < pathArray.length; j++) {
            if (validCheat(pathArray[i], pathArray[j], area)) {
                cheats.push({
                    start: pathArray[i],
                    end: pathArray[j],
                    diff: j - i - 2,
                });
            }
        }
    }
    return cheats;
}

const fastestPath = findFastestPath(area);
const cheats = findCheats(area, fastestPath);
const cheatGroups = cheats.reduce((acc, cheat) => {
    if (!acc[cheat.diff]) acc[cheat.diff] = [];
    acc[cheat.diff].push(cheat);
    return acc;
}, {})
const groupCounts = Object.keys(cheatGroups).reduce((acc, key) => {
    acc[key] = cheatGroups[key].length;
    return acc;
}, {});

const savesToBeChecked = 20;
const cheatWithXSaves = Object.keys(groupCounts).reduce((acc, key) => {
    const saves = parseInt(key);
    if (saves >= savesToBeChecked) {
        acc += groupCounts[saves];
    }
    return acc;
}, 0);
[fastestPath.size, cheatWithXSaves]

[ 85, 5 ]

In [5]:
// Part 2 - How many cheats would save you at least 100 picoseconds with cheats last 20 steps?

const findCheatsV2 = (area: string[][], path: Set<string>) => {
    const cheats = [];
    const pathArray = [...path].map((p) => p.split("-").map(Number));
    for (let i = 0; i < pathArray.length - 1; i++) {
        for (let j = i + 1; j < pathArray.length; j++) {
            const distance = Math.abs(pathArray[i][0] - pathArray[j][0]) + Math.abs(pathArray[i][1] - pathArray[j][1]);
            if (distance <= 20 && distance < j - i) {
                cheats.push({
                    start: pathArray[i],
                    end: pathArray[j],
                    diff: j - i - distance,
                });
            }
        }
    }
    return cheats;
}

const cheatsV2 = findCheatsV2(area, fastestPath);
const cheatGroupsV2 = cheatsV2.reduce((acc, cheat) => {
    if (!acc[cheat.diff]) acc[cheat.diff] = [];
    acc[cheat.diff].push(cheat);
    return acc;
}, {})
const groupCountsV2 = Object.keys(cheatGroupsV2).reduce((acc, key) => {
    acc[key] = cheatGroupsV2[key].length;
    return acc;
}, {});

const savesToBeCheckedV2 = 50;
const cheatWithXSavesV2 = Object.keys(groupCountsV2).reduce((acc, key) => {
    const saves = parseInt(key);
    if (saves >= savesToBeCheckedV2) {
        acc += groupCountsV2[saves];
    }
    return acc;
}, 0);
[fastestPath.size, cheatWithXSavesV2]

[ 85, 285 ]